In [1]:
class mymodel(object):
    def __init__(self,batch_size = 128,ncol = 180, n_hidden = 4, hidden = [100,10,10,10]):
        self.batch_size = batch_size; self.ncol = ncol
        self.n_hidden = n_hidden; self.hidden = hidden
        self.gg = tf.Graph()
        with self.gg.as_default():
            self.speed = tf.placeholder(tf.float32);speed = self.speed
            self.Xs = [];Xs = self.Xs
            self.Ys_ = [];Ys_ = self.Ys_
            self.brs = [];brs = self.brs
            for i in range(batch_size):
                Xs.append(tf.placeholder(tf.float32, shape = [None,ncol]))
                Ys_.append(tf.placeholder(tf.float32, shape = [None]))
                brs.append(tf.placeholder(tf.float32, shape = [None]))

            self.W = [];self.b = []
            hidden = [ncol]+hidden
            for i in range(n_hidden):
                self.W.append(tf.Variable(tf.truncated_normal([hidden[i],hidden[i+1]])))
                self.b.append(tf.Variable(tf.ones([hidden[i+1]])/10.0))
            self.W.append(tf.Variable(tf.truncated_normal([hidden[-1]])))
            self.b.append(tf.Variable(tf.ones([1])/10.0))
            
            W = self.W;b = self.b
            cross_entropys = []
            Ys = []
            correct_prediction = []
            trick = []
            for i in range(batch_size):
                size = tf.shape(Xs[i])[0]
                tmp = Xs[i]
                for ii in range(n_hidden):
                    tmp = tf.nn.relu(tf.matmul(tmp,W[ii])+b[ii])

                Ylogits = tf.einsum('jk,k->j',tmp,W[-1])+b[-1]
                Y = tf.nn.softmax(Ylogits)
                Ys.append(Y)
                #trick_loss = tf.nn.l2_loss(tf.nn.sigmoid(Ylogits)-brs[i])
                trick_loss = -tf.reduce_mean(tf.log(self.Ys_[i]+1e-4) * Y)

                correct_prediction.append(tf.equal(tf.argmax(Y, 0), tf.argmax(Ys_[i], 0)))
                log_size = tf.log(tf.cast(size,tf.float32))+1.0
                cross_entropys.append(tf.nn.softmax_cross_entropy_with_logits(logits = Ylogits, labels = Ys_[i])/log_size)

                trick.append(trick_loss)
                #cross_entropys.append(-tf.reduce_mean(Ys_[i] * tf.log(Y)))
            total = np.sum(cross_entropys)/batch_size
            trick_sum = np.sum(trick)

            self.all_loss = total+ 0.0 * trick_sum;all_loss = self.all_loss
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

            self.vcts = tf.placeholder(tf.float32, shape = [None,ncol])
            tmp = self.vcts
            for ii in range(n_hidden):
                tmp = tf.nn.relu(tf.matmul(tmp,W[ii])+b[ii])
            self.pred = tf.nn.sigmoid(tf.einsum('jk,k->j',tmp,W[-1])+b[-1])

            #train_step = tf.train.AdamOptimizer(speed).minimize(total)
            self.train_step = tf.train.AdamOptimizer(speed).minimize(all_loss)
            init = tf.global_variables_initializer()
        with self.gg.as_default():
            self.sess = tf.Session(graph = self.gg)
            self.sess.run(init)
            
    def fit(self, X_train, y_train, brss, n_epoch = 10, n_iter = 1, plot = False, max_speed = 0.02, min_speed = 0.005):
        accus = []; log = []
        nstep = int(1.0 * n_epoch * X_train.shape[0]/self.batch_size)
        batch_size = self.batch_size
        for i in range(nstep):
            batch_idx = np.mod(np.arange(batch_size)+ i*batch_size,X_train.shape[0])
            fdict = {self.Xs[i]:X_train[batch_idx[i]] for i in range(batch_size)}
            fdict.update({self.Ys_[i]:y_train[batch_idx[i]] for i in range(batch_size)})
            fdict.update({self.brs[i]:brss[batch_idx[i]] for i in range(batch_size)})
            speed = min_speed + max_speed * np.exp(-i *5.0/nstep)
            fdict.update({self.speed:speed})
            for j in range(n_iter):
                if plot:
                    _, accu, loss = self.sess.run([self.train_step,self.accuracy, self.all_loss], feed_dict = fdict)
                    accus.append(accu); log.append(loss)
                else:
                    self.sess.run(self.train_step, feed_dict = fdict)
        if plot:
            from matplotlib import pyplot as plt
            print accus
            plt.plot(accus, color = 'red')
            plt.plot(log/max(log), color = 'blue')

    def predict(self, X):
        return self.sess.run(self.pred, feed_dict = {self.vcts:X})
        

In [2]:
def prepare(df, prer):
    demo = prer
    demo.initiate()
    tmp1 = demo.fillna(df)
    tmp2 = demo.transform(tmp1)
    ready = demo.dropcols(tmp2)    
    return ready

def reshape(table, ordid):
    array = table.groupby(ordid).apply(np.array)
    return array.values

def mytrick(label, ordid, trickid):
    table0 = label.groupby([ordid,trickid]).transform(lambda x: np.ones(len(x))*1.0 if x.max() == 1 else x)
    res = table0.groupby(ordid).apply(np.array)
    return res
    
def get_ready(df):
    ready = prepare(df)
    ready = ready.drop('orderlabel',axis = 1)
    X = reshape(ready, df['orderid'])
    y = reshape(df['orderlabel'],df['orderid'])
    #hotel_y = trick(df['orderlabel'], ordid = df['orderid'], trickid = df['hotelid'])
    broom_y = mytrick(df['orderlabel'], ordid = df['orderid'], trickid = df['basicroomid'])
    return X,y,broom_y
    
def predict_proba(df, model):
    input = prepare(df)
    proba = model.predict(input.values)
    #proba = model.sess.run(model.pred, feed_dict = {model.vcts:input.values})
    result = df.loc[:,['orderid','roomid']]
    result['predict_proba'] = proba
    return result

def final_pred(data, key):
    res = pd.DataFrame()
    res['predict_roomid'] = data.groupby('orderid').apply(lambda x: x.sort_values(key, ascending = False)['roomid'].values[0])
    res.reset_index(inplace = True)
    return res